In [68]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)

Versão do TensorFlow:  2.16.1


In [69]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [70]:
# Carregar os dados do arquivo .pkl
with open('../../datas/angle_datas.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [71]:
%load_ext tensorboard
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(3, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/100


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 842ms/step - accuracy: 0.7623 - loss: 2.3938 - val_accuracy: 0.7775 - val_loss: 1.6314
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7608 - loss: 1.8473 - val_accuracy: 0.4075 - val_loss: 1.8456
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3692 - loss: 2.0852 - val_accuracy: 0.4827 - val_loss: 2.1554
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5116 - loss: 2.2555 - val_accuracy: 0.7746 - val_loss: 1.2412
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.7601 - loss: 1.4473 - val_accuracy: 0.7775 - val_loss: 1.3147
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.7608 - loss: 1.5176 - val_accuracy: 0.7775 - val_loss: 1.3896
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.7608 - loss: 1.5836 - val_accuracy: 0.7775 - val_loss: 1.4351
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.7608 - loss: 1.6180 - val_accuracy: 0.7775 - val_loss: 1.4515
Ep

In [72]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8467 - loss: 0.2743
Model Accuracy: 84.06%


In [73]:
# Ações
# 0 Nothing
# 1 Girar para a Esquerda
# 2 Girar para a Direita

# GO TO RIGHT
angle = -33
 
class_names = ['Nothing', 'RoteteLeft', 'RotateRight', ]
predictions = model.predict(np.array([ angle]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2-Must be GO TO RIGHT: RotateRight - with a 39.42 percent confidence.


In [74]:
%tensorboard

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [75]:
model.save("../../models/angle/model_saved.h5")
model.save(filepath="../../models/angle/model_saved.keras")


 

### From here to you only MACOS to test 

In [76]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/angle/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/angle/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/angle/saved_model/' ../../models/tf/angle/web_model

INFO:tensorflow:Assets written to: ../../models/angle/saved_model/assets


INFO:tensorflow:Assets written to: ../../models/angle/saved_model/assets
